In [1]:
print("hello")

hello


In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# Columbia EECS E6893 Big Data Analytics
"""
This module is used to pull data from twitter API and send data to
Spark Streaming process using socket. It acts like a client of
twitter API and a server of spark streaming. It open a listening TCP
server socket, and listen to any connection from TCP client. After
a connection established, it send streaming data to it.


Usage:
  If used with dataproc:
    gcloud dataproc jobs submit pyspark --cluster <Cluster Name> twitterHTTPClient.py

  Make sure that you run this module before you run spark streaming process.
  Please remember stop the job on dataproc if you no longer want to stream data.

Todo:
  1. change the credentials to your own

"""

import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
import socket
import json


# credentials
# TODO: replace with your own credentials
ACCESS_TOKEN = '1447557346117525509-dCFV0j1C35r0hx1SnyfJhW68wFt0RO'     # your access token
ACCESS_SECRET = 'zCwGbv02uZC0qbuBupzt9T9RrZnYCrXvLALhjzyPsFJX7'    # your access token secret
CONSUMER_KEY = 'rrXC3A46lGeSsHi6Yg6pDjtUa'     # your API key
CONSUMER_SECRET = '5CGdbrzApQCnsMHKqoJhOKRNj9XnfJuvYfJdJKs0MhNlBGD7e0'  # your API secret key
Global_tweet = []
# the tags to track
tags = ['#', 'bigdata', 'spark', 'ai', 'movie']

class TweetsListener(StreamListener):
    """
    tweets listener object
    """
    def __init__(self, csocket):
        self.client_socket = csocket
    def on_data(self, data):
        try:
            msg = json.loads( data )
            print('TEXT:{}\n'.format(msg['text']))
            Global_tweet.append(msg['text'])
            self.client_socket.send( msg['text'].encode('utf-8') )
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
            from pyspark.sql import Row
            rdd = sc.parallelize(Global_tweet)
            row_rdd = rdd.map(lambda x: Row(x))
            df=sqlContext.createDataFrame(row_rdd,['tweets'])
            df.coalesce(1).write.csv("gs://mk4427hw2/hadoop/tmp/bigquery/pyspark_output/lda_ouput/lda/")
            print("Wrote to csv")
            return False
        # return True
    def on_error(self, status):
        print(status)
        return False

def sendData(c_socket, tags):
    """
    send data to socket
    """
    auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    twitter_stream = Stream(auth, TweetsListener(c_socket))
    twitter_stream.filter(track=tags,languages=['en'])


class twitter_client:
    def __init__(self, TCP_IP, TCP_PORT):
        self.s = s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.s.bind((TCP_IP, TCP_PORT))

    def run_client(self, tags):
        try:
            self.s.listen(1)
            while True:
                print("Waiting for TCP connection...")
                conn, addr = self.s.accept()
                print("Connected... Starting getting tweets.")
                sendData(conn,tags)
                conn.close()
        except KeyboardInterrupt:
            exit


if __name__ == '__main__':
    client = twitter_client("localhost", 9001)
    client.run_client(tags)


Waiting for TCP connection...
Connected... Starting getting tweets.
TEXT:30分だけ パスはいつもの / お部屋入ったらコメント下さい〜！ https://t.co/1Kezvsg7ee

TEXT:RT @OT9TRANS: 211025 🍑💬

MM: Ohh everyone is surprised because the quality is good?ㅋㅋㅋㅋ it’s like a movie right?! I like it too because it’…

TEXT:RT @OT9TRANS: 211025 🍑💬

MM: Ohh everyone is surprised because the quality is good?ㅋㅋㅋㅋ it’s like a movie right?! I like it too because it’…

TEXT:Two full pages of notes for one issue. I'm feeling very good about this. 🥰😌🥰 https://t.co/RB3Sb0SnGx

TEXT:RT @decepcionadamas: tr4nsar eu não sei  mas se quiser tirar um cochiIo a tarde toda  ou assistir desenho   tamo ai

TEXT:RT @OT9TRANS: 211025 🍑💬

MM: Ohh everyone is surprised because the quality is good?ㅋㅋㅋㅋ it’s like a movie right?! I like it too because it’…

TEXT:RT @iJaadee: A Generator can always keep their life flowing in a helpful direction when they remember to focus on what things they are exci…

TEXT:@JHunterPearson Happy birthday! The best thing 

TEXT:THIS LOOKS LIKE A MOVIE WTF

TEXT:Les lunettes de soleil elles déforment la vision j’ai l’impression que j’ai pris des champis

TEXT:J’ai envie de manger 3 Mcflurry m&amp;m’s coulis caramel là

TEXT:RT @OT9TRANS: 211025 🍑💬

MM: Ohh everyone is surprised because the quality is good?ㅋㅋㅋㅋ it’s like a movie right?! I like it too because it’…

TEXT:Cotton digital print cushion cover Lovely girl print Pillow cover Bed decor Sofa decor Decorative Cushion cover Hom… https://t.co/W8UwZ7O69C

TEXT:RT @WinmetawinO: 📢 Twitter Trending 25.10.21 

📍Lazada 11.11
⏰09.30 น.
Hashtag 👇
# Lazada1111ถูกสุดในรอบปี

📍Simplus Super Brand Day
⏰ 11.0…

TEXT:RT @jhoprs: hobi? ♥︎ hobi &lt;3 ! hobi ! ji &amp; ! hobie ! # hobi ★ hobie &amp; my hobie ☹︎ i love you hobi♡ https://t.co/XVCgXV50bQ

TEXT:RT @MRevoilou: @ONPPDC1 @FreyjaCharlie @Natio_MLaFrance @TomS82794181 @4Tchat @AELTERMANbecas1 @LIMONIER1 @PartisansMarine @ZemmourEric Au…

TEXT:@itzjacton @yangasc1935 Movie gani hili

TEXT:RT @hinataa69: les homm

TEXT:RT @barefootchic: My 17yr old daughter just said “if one person dies from a prop gun in a movie set, everyone talks about its safety relent…

TEXT:RT @iamrajesh_sct: Most Probably #Valimai the Only Movie Which Release On Pongal 2022 From Tamil Cinema...

TEXT:RT @ticorilofreire: acho engraçado que quando saiu o ** todo mundo amou mas aí como não supriu as expectativas que vcs criaram de chart e p…

TEXT:@tokkyoto baixa ai pra nos jogar po

TEXT:@MrSexyDonut Oui mais je veux pas faire de sous jobs j’ai 29 ans si je pars quelque part c’est a minima pour un truc en bureau

TEXT:RT @EnModeMacaron: SVP ne RT plus cette vidéo d’Agnès Pannier-Runacher que les macronistes veulent cacher tellement ils ont honte!😂

Depuis…

TEXT:RT @WarpsiwaHOT: Nenne Ui
Full Movie https://t.co/80HVJwCwUN

🌟เล่นเสียได้เงินคืนสูงกว่าใคร
🌟น้องใหม่รับโบนัส60%
🌟ฝาก1บาทก็รวยได้แล้ว

⚡️#…

TEXT:gosto mt dela mas toda vez ela fala alguma coisa assim e depois diz “ai gente desculpa digitei erradokkkk”

ta forç… htt

TEXT:RT @soysouce_2255: 命令口調で襲われながらカウント責め。 / 懐いてる後輩は本当はドSで... https://t.co/Pl8bY1dRzf

TEXT:RT @soysouce_2255: 命令口調で襲われながらカウント責め。 / 懐いてる後輩は本当はドSで... https://t.co/Pl8bY1dRzf

TEXT:Movie script rough draft lines

Awwwwwe!!!

Lil ghurlz got idea notebooks

Designing outlines for lifetime rounds… https://t.co/GAeD7AocDC

TEXT:RT @sen_sen_sen_sen: セ　ほ少 https://t.co/4BtbYlGKWd

TEXT:RT @sawiisya: J’ai pratiquement aucune visibilité mais je voulais juste vous montrer les pépites que fait ma copine 🤌🏾 https://t.co/MRRhlQX…

TEXT:RT @Alextrxm: ce béluga qui joue au ballon avec un homme c’est la chose la plus incroyable que j'ai vu aujourd’hui, c’est trop mignon 😂❤🐳…

TEXT:Je ne viens pas de l’Est de Montréal, mais ce que j’ai remarqué en déménageant là-bas, c’est que c’est complètement… https://t.co/odDCmeNYkU

TEXT:RT @MzBuxton7: PROVERBS SUNDAY! The fear of man brings a snare: but whoever puts his trust in the Lord shall be safe. 
In all our pursuit,…

TEXT:wAllah a cause de ça. J’ai failli m

TEXT:RT @qiqicutiepie: จื่อวีซีนนี้เหมือน ai มาก สวยจึ้งสวยตาพร่าเสื้อผ้าหน้าผมเมคอัพไม่แกง 10 10 10 https://t.co/xT89eCzyRT

TEXT:RT @soysouce_2255: 命令口調で襲われながらカウント責め。 / 懐いてる後輩は本当はドSで... https://t.co/Pl8bY1dRzf

TEXT:# TWICE PLEASEE

TEXT:RT @movie_theaters: Alamo Drafthouse, The Godfather Of Dinner-And-A-Movie Theaters, Finally Opens In Manhattan https://t.co/GfKyiC9noB via…

TEXT:RT @Hazelchousana: โอ้ย น้องโมะมาถามวั้นว่าดูทริลเลอร์ยัง แล้ววั้นแปลกใจเพราะว่าคุณภาพทริลเลอร์งานดี ดีสุดที่เคยทำมาเลย น้องโมะขำเลย5555555…

TEXT:RT @ALittleBraver92: a sight for sore eyes #Jin #진 #석진 #김석진 # https://t.co/3Cf8vKpI4U

TEXT:RT @Jacemandaspace2: The general audience is smart enough to be able to handle two full time batmen. @wbpictures already knows that bc they…

TEXT:@gardennshed Para ai mano to de ressaca

TEXT:RT @satokun_osato: p.h. / 幻の歌枠 https://t.co/tKqSETUKV6

TEXT:RT @RealShotgunSuge: ALBUM LOADING ……. What Ya Life Like THE MOVIE https://t.co/3PpDirtsdp

TEXT:RT @O712page: เห็นฉากนี้

...

TEXT:J’ai retrouver ma comoco 🥰🇰🇲 @meufDicietlabas au lac ma voisine de 3min wsh subhan الله je l’ai connu à la mosquée

TEXT:RT @LILITEAMTH327: [IG UPDATE] Bvlgari

🗓 24.10.2021

🔗https://t.co/u5nRcSYOqN

블랙핑크 리사 # LISA #LALISA #LALICE #BLACKPINK #리사 #블랙핑크 https:/…

TEXT:モイ！iPhoneからキャス配信中 - / すこしだけ https://t.co/bmAxuKRhYr

TEXT:Ini lucu😁😁😁😁

TEXT:RT @YassinB__: J’ai revu Scary Movie hier soir, et cette scène reste la meilleure de toute la saga 

Pas de débat possible https://t.co/Qm5…

TEXT:RT @leffisdepute: Ptdr j'ai envie de mourir 🤣🤣 https://t.co/N995mRPaN5

TEXT:RT @Phiyen1988: JUNGKOOK GỬI ĐẾN HATER TRONG CONCERT HÔM NAY

Jungkook: Mình có rất nhiều điều muốn phàn nàn. Mình cũng có đôi lời muốn gửi…

TEXT:https://t.co/geBtnePEB2

TEXT:RT @AwKaren91: GA time! Special ultah Lee Know &amp; Ayah Eren (suami saya)😁 
Gift utk 2 org :
1.NoEasy Album only+PC ChanMin
2.DS Seungmin+Fra…

TEXT:RT @barefootchic: My 17yr old daughter just said “if one person dies from a prop gun in a movie se

TEXT:RT @OT9TRANS: 211025 🍑💬

MM: So truthfully I'm in the cosmos right now...🔋🚀🚀🚀
But I couldn't say anything until now..because it'd be a spoi…

TEXT:RT @travel_dx: ＼ VR Tripリリース記念キャンペーン🎉 ／

AI自動翻訳付きVR旅行体験 #VRTrip の公式リリースを記念して、
抽選で100名様にOculus quest2またはAmazonギフトコードプレゼント🎁✨

🌴応募方法
①@travel_…

TEXT:RT @PRIMESELECTION8: # 93 https://t.co/wG0pZa9qZP

TEXT:RT @DrPacoMoreno1: Reino Unido tiene un alto # en nuevos casos de #COVID19, pero no así en enfermedad grave y fallecimientos.
* Las vacunas…

TEXT:Música nova eu acho viu

TEXT:@euluanals AI O ALEX CHAMA PRECISA DE UMA PEDIATRA P CIRUGIA, E CHAMA A ARIZONA,  VOLYA ELA E A CALLIE

TEXT:?

TEXT:https://t.co/hDBVKvz1zu

TEXT:Iced coffee + good movie + esok cuti 🥰 I love things that rhyme 🤧

TEXT:RT @Seba_Terry: I was watching a movie with my straight friend and this happened… 🎞️🔥
Just for this month 🔥60%📴at https://t.co/6d6tK3cCXh o…

TEXT:@thomasfuyu @Taebumjhyun kxjsd je te l’avais dis !! mais là j’ai lu moins vite sinon j’aurais fini dan

TEXT:RT @popjohnpol: https://t.co/FgLHw92DBP

#WomenWhoCode #javascript #GirlsWhoCode #Ronaldo #messi #PyTorch #NFT #Python #programming #projec…

TEXT:@RVTAEE J'ai grave la flemme mais je peux

TEXT:@k_na0406 it hurts /? huhu im sad over a movie hahaha 😭💔

TEXT:@mirko_soumise J'ai l'impression que je vais fondre*
*Mon corps chauffe*

TEXT:RT @braulioesca: Añañai domingo de Carmita asada “El Asa’O” deja la cocina y pasa más tiempo disfrutando con la familia y amigos… pedidos W…

TEXT:OMGGGGG DUNE SUPREMACY

TEXT:RT @pezoutlaw: The Pez Outlaw Movie/Documentary is Finished. https://t.co/I9MwsviGoq

TEXT:RT @sougatilhos: as vezes me da vontade  de afastar de todo mundo e ficar sozinha  ai lembro que já to sozinha

TEXT:@MoMoAoChan Let’s go!!! Everyone gonna go to the movie theater dressed in suits or dresses to attend their wedding 😆😆

TEXT:ai chaeyoung te entendo ele é muito lindin

TEXT:Nell'Angelus di Piazza S. Pietro, @Pontifex si è rivolto oggi direttamente ai migranti in #Libia rico

TEXT:KHÔNG...LÀ Ý TƯỞNG CỦA AI 😭😭😭😭😭😭😭😭😭😭😭

TEXT:RT @WarpsiwaHOT: Shirato Hana
Full Movie https://t.co/mYipCHnf7A

🌟เล่นเสียได้เงินคืนสูงกว่าใคร
🌟น้องใหม่รับโบนัส60%
🌟ฝาก1บาทก็รวยได้แล้ว…

TEXT:Clueless
It’s Complicated
Legally Blonde
Father of the Bride
The Incredibles 
Home Alone (1&amp;2)
First Wives Club
The… https://t.co/y0tmX74rO0

TEXT:RT @WarpsiwaHOT: Shirato Hana
Full Movie https://t.co/B2wKnGp4Lp

🧧AFB9 🧧
🧧คืนยอดเสียมากที่สุดในไทย🧧

✅คืนยอดเสีย7%ได้คืนทุกสัปดาห์ 
✅ฝาก-ถ…

TEXT:RT @NCSounds: We thought we'd put an NCS twist to some classic scary movie posters. Here's Friday The 13th... but make it @UnknownBrain_ 💀…

TEXT:Le match du classico est nuul  #우리집_큰고양이_리노_생일 tsais le # qui a rien a voir cjxjdkdk 🍣🍚

TEXT:RT @sionrite: # 57 https://t.co/2v4dQYcegC

TEXT:RT @BotThiendia: AE ai mua clip anh Trai Gymer địt chị gái khẩu dâm FULL DH dài hơn 9 phút thì inbox cho em nhé ☺️
Hoặc Zalo: 0368595348 @B…

TEXT:RT @tahsandes: สปอย # underthegreenlight ตอน29 แบบสั้นๆ

TEXT:#スーパーポイントス

TEXT:RT @VictoriaJustice: Can’t believe it’s been 5 years since this came out 🤯 I’ve been a fan of this movie since I was in 5th grade, so it wa…

TEXT:RT @LaNissarda: #Normandie : Le maire d’un petit village agressé alors qu’il intervenait pour mettre fin à un rodéo urbain. « J’ai reçu un…

TEXT:RT @lbrgt_: envie d’un chinois mtn

TEXT:Los Papeles de Pandora en Atlas Network https://t.co/nZoaTPlHd3 # via @JulianMaciasT

TEXT:RT @OwensDamien: Saw Dune last night. Man, that is one sandy movie. Up there with the two sandiest movies of all time, Lawrence of Arabia a…

TEXT:Is consumption-based IT a good option for your organization? | #AI #Cloud https://t.co/5UMaBJXocy

TEXT:RT @popjohnpol: https://t.co/FgLHw92DBP

#WomenWhoCode #javascript #GirlsWhoCode #Ronaldo #messi #PyTorch #NFT #Python #programming #projec…

TEXT:RT @PJ_TAN: ฮ่าๆๆๆๆๆอิสั้นต่างตกใจที่งานออกมาดี

TEXT:RT @sbhjade: cc j’ai réalisé mon premier fanart de kurapika ! https://t.co/0O94t1yO6A

TEXT:@likhiteshNBK Along with S